# Operation Research Fall 99 <img src = 'https://ece.ut.ac.ir/cict-theme/images/footer-logo.png' alt="Tehran-University-Logo" width="150" height="150" align="right">
## Assignment 1 : Sudoku
### Dr. Mohammad Shokri
### By Omid Vaheb

### Question:
In this assignment I modeled Sudoko game as an optimization problem and solved it using pulp library of python.

### Sudoko:
Sudoku is known as a puzzle of numbers, is a puzzle based on logic. The goal of this game is to add numbers from 1 to 9 in a 9×9 table which includes 9, 3×3 tables called regions. At the start, numbers are preprepared in the table. Every region, Row, and column must include numbers from 1 to 9 without repetition.

### Modeling:
If you can formulate a problem in terms of a linear objective function and linear inequality constraints, linear programming (LP) is a powerful tool for finding its optimal solutions. In this problem we can use ILP or integer linear programming which is a linear programming in which its variables are all integers. 

##### Objective Function:
Since there is no solution to a sudoku that is better than another, I define objective function with zero value and the real challenge is to find any feasible set.

##### Linear Constraints:
Sudoku consists of filling in a nine-by-nine grid, such that each row, column, and three-by-three subgrid contains only one of an integer value. Since there are nine values each box can be filled with, and we are constrained along each of the nine rows and nine columns, we have 9³ = 729 parameters. Below we will index these values in x by v (value), r (row), and c (column).
More concretely, we can formulate our objective function and linear constraints as follows:

<img src = 'https://miro.medium.com/max/576/1*DvDxo0HQ0XkP7tmIiYOIGQ@2x.png' alt="formula" width="800" height="800" align="center">

The first constraint requires that each cell contains one value. The second and third constraints maintain columns and rows conditions. The last constraint fixes that only one of a value is found in each subgrid. The original starting state of each non-empty cell is the last constrait of this problem.

Now we implement our solution for this problem in python. The first step is to install and import required library.

In [1]:
#!pip install pulp
from pulp import *

Now we define available values for each cell an also define boxes or sub-grids for ease of access along the solution.

In [2]:
Vals = ["1", "2", "3", "4", "5", "6", "7", "8", "9"]
Rows = ["1", "2", "3", "4", "5", "6", "7", "8", "9"]
Cols = ["1", "2", "3", "4", "5", "6", "7", "8", "9"]

Boxes = []
for i in range(3):
    for j in range(3):
        Boxes += [[(Rows[3 * i + k], Cols[3 * j + l]) for k in range(3) for l in range(3)]]

The next step is to create variables of problem and defining objective function.

In [3]:
prob = LpProblem("Sudoku Problem", LpMinimize)
choices = LpVariable.dicts("Choice", (Vals, Rows, Cols), 0, 1, LpInteger)
prob += 0, "Arbitrary Objective Function"

D:\Anaconda\lib\site-packages\pulp\pulp.py:1198: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


Now we define the constraint ensuring that only one value can be in each square is created.

In [4]:
for r in Rows:
    for c in Cols:
        prob += lpSum([choices[v][r][c] for v in Vals]) == 1, ""

Row, column and box constraints are also added for each value.

In [5]:
for v in Vals:
    for r in Rows:
        prob += lpSum([choices[v][r][c] for c in Cols]) == 1,""
        
    for c in Cols:
        prob += lpSum([choices[v][r][c] for r in Rows]) == 1,""

    for b in Boxes:
        prob += lpSum([choices[v][r][c] for (r,c) in b]) == 1,""

The final step before solving the problem is to define constraints regarding starting values in cells. We read them from a text file and add them to problem.

In [6]:
# The starting numbers are entered as constraints. Note: choices is indexed by value, row, column.
f = open("input.txt", "r")
N = int(f.readline())
A = []
for i in range(N):
    A.append([m for m in f.readline().split()])
f.close()
for k in A:
    prob += choices[k[0]][k[1]][k[2]] == 1,""

The last step is to solve problem using pulp and write it in a file.

In [7]:
prob.solve()
output = open('output.txt','w')
for r in Rows:
    temp = ""
    if r == "1" or r == "4" or r == "7":
        output.write("+-------+-------+-------+\n")
        print("+-------+-------+-------+")
    for c in Cols:
        for v in Vals:
            if value(choices[v][r][c]) == 1:       
                if c == "1" or c == "4" or c =="7":
                    output.write("| ")
                    temp += "| "
                output.write(v + " ")
                temp += v + " "
                if c == "9":
                    output.write("|\n")
                    temp += "|"
    print(temp)
output.write("+-------+-------+-------+")
print("+-------+-------+-------+")
output.close()
print("Solution is also written to output.txt")

+-------+-------+-------+
| 1 8 4 | 5 6 3 | 9 7 2 |
| 6 2 7 | 1 9 8 | 5 3 4 |
| 5 9 3 | 7 2 4 | 8 1 6 |
+-------+-------+-------+
| 4 7 9 | 2 3 5 | 6 8 1 |
| 3 5 8 | 6 7 1 | 4 2 9 |
| 2 1 6 | 4 8 9 | 7 5 3 |
+-------+-------+-------+
| 9 6 5 | 8 1 2 | 3 4 7 |
| 8 3 1 | 9 4 7 | 2 6 5 |
| 7 4 2 | 3 5 6 | 1 9 8 |
+-------+-------+-------+
Solution is also written to output.txt
